In [1]:
### hello

#OLD DATA PREPPING
'''
#Logan Kelsch + JJ
'''
'''
/*
 *  taking this version of neural network training takes only a few features
 *          [ high, low, open, close, volume, TimeOfDay, DayOfWeek ]
 *  and uses common techniques described in 'features_creation.py' to largely
 *  expand dimensionality. 
 *  As of writing this (11/24/24) there is limited expansion
*/
''' 
'''
#IMPORT LIBRARIES-------------------------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from keras.optimizers import SGD
from keras.initializers import GlorotUniform
from keras.initializers import RandomNormal
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from funcs_data_process import *
from feature_creation import *
from performance_printout import *
from tensorflow.keras.optimizers import AdamW


# Load the dataset
data = pd.read_csv('betaset_tmp.csv')

#---------------#
    MODEL VARIABLES
#---------------#


indp_size       = 0.05       #size of indepentendent set of samples
test_size       = 0.15       #size of validation set of samples
num_isol_feats  = 0         #number of features to be left out of PCA transformation
                            #   this function is depricated/needs updated
use_PCA         = False      #bool for use of PCA feature decomp. and transformation
comps_PCA       = 128        #number of primary components to use under PCA t.

time_steps      = 5        #LSTM time steps used

t_start         = 570       #start time for time based sample filtering         570 is 9:30am EST
t_end           = 720       #end   time for time based sample filtering         645,720 are 10:45am,12:00pm EST

params = model_params()
params = get_model_params(
    m_type          = 'Classification'      # {'Regression', 'Classification'}

   ,target_time     = 5        #how many minutes in the future is the target
   ,c_split_val     = 5
   ,c_class_cnt     = 2
)


#drop unused target columns
data = set_target(data, params)

#collect all sample indices to be kept through time filter
#two seperate functions to keep main and ind test isolated
keep_ndx = grab_wanted_times(data.values, t_start, t_end, time_steps)

data = data.drop(columns=return_name_collection())
#data = data.drop(columns=fn_orig_time())

#confirmation printout of all features/targets
Xfeatures = data.columns[:-1]
Yfeatures = data.columns[-1]
print("TESTED FEATURES: ")
print(Xfeatures)
print("TESTING FOR: ")
print(Yfeatures)


# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

params = update_class_weights(y, params)

if(params.model_type=='Classification'):
    y, label_encoder = y_preprocess(params, y)

X_nrm = normalize_from_tt_split(X, X, test_size)
if(use_PCA):
    X = reform_with_PCA_isolated(X_nrm, X_nrm, test_size, num_isol_feats, comps_PCA)
#X, y = reformat_to_lstm(X, y, time_steps)

print('\nX shape == {}.'.format(X.shape))
print('y shape == {}.\n'.format(y.shape))


print(f'Raw Sample Count:\t{len(X)}')
X,     y     = filter_times(X,     y,     keep_ndx)
print(f'Remaining Sample Count:\t{len(X)}')

#split all samples 3 ways into training, and testing
#   and split all testing into validation and independent
X_train, X_val, X_ind, y_train, y_val, y_ind =\
    split_into_train_val_ind(X, y, test_size, indp_size, time_steps)

if(params.model_type=='Classification'):
    inv_trn_y = label_encoder.inverse_transform(y_train)
    params = update_class_weights(inv_trn_y, params)

print('\nX_train shape == {}.'.format(X_train.shape))
print('y_train shape == {}.'.format(y_train.shape))
print('X_val shape == {}.'.format(X_val.shape))
print('y_val shape == {}.'.format(y_val.shape))
print('X_ind shape == {}.'.format(X_ind.shape))
print('y_ind shape == {}.\n'.format(y_ind.shape))


'''